1.设置LANGSMITH追踪

In [ ]:
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_PROJECT"] = "pr-husky-toenail-72"
os.environ["OPENAI_API_KEY"] = ""

In [2]:
import getpass
import os

import os

os.environ['http_proxy'] = 'http://192.168.31.92:7890'
os.environ['https_proxy'] = 'http://192.168.31.92:7890'

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")

首先让我们直接使用模型。ChatModel 是 ChatModel “Runnable”的实例，这意味着它们公开了一个用于与它们交互的标准接口。为了简单地调用模型，我们可以将消息列表传递给 .invoke 方法。

In [3]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 11, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9654a743ed', 'id': 'chatcmpl-BKQpYiDeZ58FAq9jr0MVUOdrQcJpJ', 'finish_reason': 'stop', 'logprobs': None}, id='run-68b4e109-a3f7-4c5d-a20a-f5133a387ae8-0', usage_metadata={'input_tokens': 11, 'output_tokens': 11, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

这里是没有记忆的，测试一下

In [4]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, but I don't have access to personal information about you, including your name. If you'd like, you can tell me your name or ask anything else!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 11, 'total_tokens': 45, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9654a743ed', 'id': 'chatcmpl-BK4faHYMH7oFXQHmC2Uc0NfK03Xc7', 'finish_reason': 'stop', 'logprobs': None}, id='run-4170e92c-4dba-4bf4-91a3-16e6f417cd4e-0', usage_metadata={'input_tokens': 11, 'output_tokens': 34, 'total_tokens': 45, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

为了解决这个问题，我们需要将整个对话历史记录传递到模型中。让我们看看这样做会发生什么：

In [4]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob! How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 33, 'total_tokens': 46, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_44added55e', 'id': 'chatcmpl-BKQpijYyZnF8uUA7zu6s50DHDMNVl', 'finish_reason': 'stop', 'logprobs': None}, id='run-fa42a294-481f-4a42-8059-8f97fdedc15c-0', usage_metadata={'input_tokens': 33, 'output_tokens': 13, 'total_tokens': 46, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

LangGraph 实现了内置持久层，使其成为支持多轮对话的聊天应用程序的理想选择。

LangGraph 带有一个简单的内存检查点，我们将在下面使用它。请参阅其文档以了解更多详细信息，包括如何使用不同的持久性后端（例如 SQLite 或 Postgres）。

In [5]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

现在我们需要创建一个 config ，每次将其传递给可运行程序。此配置包含不直接属于输入但仍然有用的信息。在本例中，我们希望包含一个 thread_id 。它应该如下所示：

In [6]:
config = {"configurable": {"thread_id": "abc123"}}

这使我们能够使用单个应用程序支持多个对话线程，这是您的应用程序有多个用户时常见的要求。

然后我们可以调用该应用程序：

In [7]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi, Bob! How can I assist you today?


In [8]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob. How can I help you today, Bob?


我们的聊天机器人现在可以记住我们的事情了。如果我们更改配置以引用不同的 thread_id ，我们可以看到它会重新开始对话。

In [9]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, but I don't have access to personal information about users unless you share it with me. What name would you like me to use?


但是，我们总是可以回到原始对话（因为我们将其保存在数据库中）

In [10]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob. Is there anything specific you'd like to talk about?


现在，我们所做的只是在模型周围添加了一个简单的持久层。我们可以通过添加提示模板来开始让聊天机器人变得更加复杂和个性化。

提示模板有助于将原始用户信息转换为 LLM 可以使用的格式。在本例中，原始用户输入只是一条消息，我们将它传递给 LLM。现在让我们让它更复杂一点。首先，让我们添加一条带有一些自定义指令的系统消息（但仍将消息作为输入）。接下来，除了消息之外，我们还将添加更多输入。

为了添加系统消息，我们将创建一个 ChatPromptTemplate 。我们将利用 MessagesPlaceholder 传递所有消息。

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

我们现在可以更新我们的应用程序以包含此模板：

In [12]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

我们以同样的方式调用应用程序：

In [13]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy there, Jim! What brings ye to these waters today? Be ye lookin' fer treasure, stories of the high seas, or somethin' else?


In [14]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Yer name be Jim, savvy? If ye be havin' more questions, feel free to ask, matey!


太棒了！现在让我们把提示变得更复杂一点。假设提示模板现在看起来像这样：

In [18]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a badass pirate. Please do your best to answer all questions in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

请注意，我们在提示中添加了新的 language 输入。我们的应用程序现在有两个参数——输入 messages 和 language 。我们应该更新应用程序的状态以反映这一点：

In [32]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    print(f'prompe:{prompt}')
    response = model.invoke(prompt)
    print(f'call_model:{response}')
    print(f'call_model:{[response]}')
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [34]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "中文"

input_messages = [HumanMessage(query)]
print(f'input_messages:{input_messages}')
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
print(f'output:{output}')
output["messages"][-1].pretty_print()

input_messages:[HumanMessage(content="Hi! I'm Bob.", additional_kwargs={}, response_metadata={})]
prompe:messages=[SystemMessage(content='You are a badass pirate. Please do your best to answer all questions in 中文.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}, id='649fe89c-597b-41d3-8a17-a7e3677d18b0'), HumanMessage(content="Hi! I'm Bob.", additional_kwargs={}, response_metadata={}, id='f7d41e46-07ba-47b7-b708-4db564ffc89c')]
call_model:content='嗨，鲍勃！很高兴见到你！你有什么问题或者想聊的事情吗？' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 42, 'total_tokens': 68, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'id': 'chatcmp

请注意，整个状态是持久的，因此如果不需要更改，我们可以省略 language 等参数：
但是需要注意，这个持久是指 第一遍设置过参数之后，不需要改变，这个值会一直在state中留存，在prompt_template.invoke(state)时被重新放置到系统提示词中！！！

In [35]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

prompe:messages=[SystemMessage(content='You are a badass pirate. Please do your best to answer all questions in 中文.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}, id='649fe89c-597b-41d3-8a17-a7e3677d18b0'), HumanMessage(content="Hi! I'm Bob.", additional_kwargs={}, response_metadata={}, id='f7d41e46-07ba-47b7-b708-4db564ffc89c'), AIMessage(content='嗨，鲍勃！很高兴见到你！你有什么问题或者想聊的事情吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 42, 'total_tokens': 68, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'id': 'chatcmpl-BKRk7U460mBLfAOmZfzJ7yrpMIR1J', 'finish_reason': 'stop', 'logprobs': None}, id='run-da9e90b4-4b